합성곱(convolution)
- 시각적 이미지를 분석하는 데 쓰이는 인공신경망의 한 종류
- 일반적인 신경망은 이미지 데이터를 그대로 처리하는 데 반해 합성곱신경망은 이미지에서 특징을 추출해 처리한다.
- 기존 신경망에 합성곱 계층(Conv)와 풀링계층(Pooling, 풀링계층은 생략할 수 있다.)을 완전 연결한 계층 이전에 추가하여 원본 이미지에   필터링 기법을 적용한다.
- 이어 해당 이미지에 분류 연산을 수행하도록 구성한다.
- 완전연결신경망과 달리 뉴런을 필터 혹은 커널이라고도 부른다.(커널의 크기는 하이퍼파라미터이다.)
- 합성곱 계산을 통해 얻은 출력을 특성 맵(feature map)이라고 부른다.

케라스 합성곱 층

In [2]:
# Conv2D 클래스의 매개변수인 필터의 개수와 커널의 크기 지정._커널의 크기는 보통(3,3)이나 (5,5) 크기 권장.
from tensorflow import keras
keras.layers.Conv2D(10, kernel_size=(3,3), activation='relu')

패딩과 스트라이드
- 패딩: 입력 배열의 주위를 가상의 원소로 채우는것

        1. 세임 패딩  : 입력과 특성 맵의 크기를 동일하게 만들기 위해 입력 주위에 0으로 패딩하는것
                        (실제값은 0으로 채워져 있기 때문에 계산에 영향을 미치지는 않음.).
                        일반적인 합성곱 신경망에서 많이 사용됨.
                        ** 세임패딩의 경우 입력과 만들어진 특성 맵의 가로세로 크기가 같다.
        2. 밸리드 패딩: 패딩 없이 순수한 입력 배열에서만 합성곱을 하여 특성 맵을 만드는 경우.
  
- 스트라이드: 좌우, 위아래로 이동하는 이동의 크기
              스트라이드 기본은 1(한 칸씩 이동) 

In [3]:
keras.layers.Conv2D(10, kernel_size=(3,3), activation='relu', padding='same', strides=1)

풀링
- 합성곱 층에서 만든 특성 맵의 가로세로 크기를 줄이는 역할을 수행(특성맵의 개수는 줄이지 않음)
- 풀링에는 가중치가 없으며, 최댓값(최대풀링)이나 평균값(평균풀링)을 계산하는 역할을 수행함.
   1. 최대풀링(MaxPooling) : 클래스는 MaxPooling2D
   2. 평균풀링(AveragePooling) : 클래스는 AveragePooling2D
   3. 평균풀링은 특성 맵에 있는 중요한 정보를 (평균하여) 희석시킬 수 있기 때문에 최대 풀링을 많이 사용.
- 풀링크기와 스트라이드가 같으며 패딩도 없음. ( ex) 풀링의 크기가 (2,2)이면 가로세로 두칸씩 이동)


In [5]:
# 풀링의 크기 지정(대부분 풀링의 크기는 2)
keras.layers.MaxPooling2D(2)

합성곱 신경망의 전체 구조
- 1. 입력층:(4,4)입력 ->
            합성곱 층에서 사용할 커널의 크기는 (3,3)이고 세임패딩으로 1픽셀이 입력 데이터 주변에 추가. ->
            패딩이 추가된 입력에서 합성곱 수행( ex)필터 3개는 각각 (3,3)크기 가중치를 가지고 있으며 필터마다 절편이 하나씩 있음)->
            따로 입력이 없으면 합성곱의 스트라이드는 항상 1이므로 특성맵의 크기는 입력과 동일한 (4,4)임.->
            3개의 필터가 하나씩 합성곱의 출력을 만들고, 이 출력이 합쳐져서(4,4,3) 크기의 특성맵이 만들어짐 ->
            (합성곱 층은 활성화 함수로 렐루 함수를 많이 사용)  
  2. 풀링층:합성곱에서 만든 특성맵의 가로세로 크기를 줄임.(보통 (2,2) 풀링을 사용해 절반으로 줄인다.) ->
            풀링을 거친 특성 맵의 크기는 절반으로 줄었기 때문에 (2,2,3)이 됨. ->
            출력층에 전달하려면 3차원 배열을 1차원으로 펼쳐야 함.(이 배열은 12개의 원소를 가진 1차원 배열이고 출력층의 입력이 됨)->
  3. 출력층: 클래스 분류(3개의 뉴런을 두게 되면 3개의 클래스를 분류)
     
             ** 출력층에서 계산된 값은 소프트맥스 활성화 함수를 거쳐 최종 예측확률이 됨.

컬러 이미지를 사용한 합성곱
- 하나의 컬러 이미지는 너비와 높이 차원 외에 깊이 차원(또는 채널 차원)이 있음.
- 커널 배열의 깊이는 항상 입력의 깊이와 같음.
- 입력이나 필터의 차원이 몇개인지 상관없이 항상 출력은 하나의 값.
- 만약 패션MNIST 데이터처럼 흑백 이미지일 경우에는 깊이 차원이 1인 3차원 배열로 변환하여 밀집층에 전달.
- 합성곱 신경망에서 필터는 처음에는 이미지에 있는 간단한 기본적인 특징(직선, 곡선 등)을 찾고 층이 깊어질수록 다양하고 구체적인 특징   을 감지 할 수 있도록 필터의 개수를 늘리며, 너비와 높이 차원을 점점 줄여 가며 어떤 특징이 이미지의 어느 위치에 놓이더라도 쉽게 감
  지 할 수 있도록 함.